- base: https://www.kaggle.com/code/vadimkamaev/postprocessin-ensemble
- select feats by importances and 3 ensembles (no postprocess) 
- update common class again*

# common class

In [1]:
import sys
import optuna
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold, KFold

import itertools
from datetime import datetime
from sklearn.decomposition import PCA

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class TreeModelling:
    def __init__(self, config):
        self.config = config 
        
        
    def lgb_train_and_valid(self, data, target, tr_idx, val_idx, feats, split):
        
        x_tr = data[feats].iloc[tr_idx]
        x_val = data[feats].iloc[val_idx]
        y_tr = target.iloc[tr_idx]
        y_val = target.iloc[val_idx]
    
        feat_imp_df = pd.DataFrame(feats, columns = ["feat"])
        model = lgb.train(self.config["lgb"]["params"], lgb.Dataset(x_tr, y_tr), self.config["lgb"]["num_round"], valid_sets=[lgb.Dataset(x_val, y_val)], 
                      callbacks = [lgb.early_stopping(stopping_rounds=self.config["lgb"]["es_round"], verbose=True), 
                                   lgb.log_evaluation(self.config["lgb"]["verbose_eval"])]) 
        feat_imp_df["imp"+str(split)] = model.feature_importance()
        valid_pred = model.predict(x_val)
    
        return model, valid_pred, feat_imp_df
    
    def lgb_test(self, models, model_name, test):
        feats = self.config[model_name]["feats"]
        for index, model in enumerate(models):
            if index == 0:
                test_pred = model.predict(test[feats]) 
            else:
                test_pred += model.predict(test[feats]) 
        test_pred /= len(models)
        return test_pred
    

    def xgb_train_and_valid(self, data, target, tr_idx, val_idx, feats, split):
        
        x_tr = data[feats].iloc[tr_idx]
        x_val = data[feats].iloc[val_idx]
        y_tr = target.iloc[tr_idx]
        y_val = target.iloc[val_idx]

        xgb_eval = xgb.DMatrix(x_val, label = y_val)
        model = xgb.train(self.config["xgb"]["params"], xgb.DMatrix(x_tr, label = y_tr), self.config["xgb"]["num_round"], evals = [(xgb_eval, "eval")], 
                          early_stopping_rounds=self.config["xgb"]["es_round"], verbose_eval = self.config["xgb"]["verbose_eval"])
        feat_imp_df = pd.DataFrame(model.get_score(importance_type="total_gain").items(), columns =["feat", "imp"+str(split)])
        valid_pred = model.predict(xgb_eval, iteration_range=(0, model.best_ntree_limit))
    
        return model, valid_pred, feat_imp_df
    
    def xgb_test(self, models, model_name, test):
        feats = self.config[model_name]["feats"]
        dtest = xgb.DMatrix(test[feats])
        for index, model in enumerate(models):
            if index == 0:
                test_pred = model.predict(dtest, iteration_range=(0, model.best_ntree_limit))
            else:
                test_pred += model.predict(dtest, iteration_range=(0, model.best_ntree_limit))
        test_pred /= len(models)
    
        return test_pred

    
    def cb_train_and_valid(self, data, target, tr_idx, val_idx, feats, split):
        
        x_tr = data[feats].iloc[tr_idx]
        x_val = data[feats].iloc[val_idx]
        y_tr = target.iloc[tr_idx]
        y_val = target.iloc[val_idx]
        
        feat_imp_df = pd.DataFrame(feats, columns = ["feat"]) 
        if self.config["task_type"] == "classification":
            model = CatBoostClassifier(**self.config["cb"]["params"])
            model.fit(x_tr, y_tr, eval_set=[(x_val, y_val)], early_stopping_rounds=self.config["cb"]["es_round"],
                      verbose_eval = self.config["cb"]["verbose_eval"])
            valid_pred = model.predict_proba(x_val)[:,1]
        elif self.config["task_type"]:
            model = CatBoostRegressor(**self.config["cb"]["params"])
            model.fit(x_tr, y_tr, eval_set=[(x_val, y_val)], early_stopping_rounds=self.config["cb"]["es_round"], 
                      verbose_eval = self.config["cb"]["verbose_eval"])
            valid_pred = model.predict(x_val)
        
        feat_imp_df["imp"+str(split)] = model.get_feature_importance()
    
        return model, valid_pred, feat_imp_df
    
    def cb_test(self, models, model_name, test):
        feats = self.config[model_name]["feats"]
        for index, model in enumerate(models):
            if index == 0:
                if self.config["task_type"] == "classification":
                    test_pred = model.predict_proba(test[feats])[:,1]
                elif self.config["task_type"]:
                    test_pred = model.predict(test[feats])
            else:
                if self.config["task_type"] == "classification":
                    test_pred += model.predict_proba(test[feats])[:,1]
                elif self.config["task_type"]:
                    test_pred += model.predict(test[feats])
        test_pred /= len(models)
        
        return test_pred
    
    
    def cv_execute(self, model_name):
            
        models = []
        feats = self.config[model_name]["feats"]
        valid_output = np.zeros(len(self.config["target"]))
        
        if model_name == "lgb":
            func = self.lgb_train_and_valid
        elif model_name == "xgb":
            func = self.xgb_train_and_valid
        elif model_name == "cb":
            func = self.cb_train_and_valid
        
        ####
        cv_scheme = MultilabelStratifiedKFold(5, shuffle=True, random_state=42).split(self.config["train_data"],
                                                                                      train_stratify)
        ####
    
        for split, (tr_idx, val_idx) in enumerate(cv_scheme): 
            if split == 0:
                model, valid_pred, feat_imp_df = func(self.config["train_data"], self.config["target"], tr_idx, val_idx, feats, split)
            else:
                model, valid_pred, tmp_imp_df = func(self.config["train_data"], self.config["target"], tr_idx, val_idx, feats, split)
                feat_imp_df = pd.merge(feat_imp_df, tmp_imp_df, on = "feat", how = "inner")
            valid_output[val_idx] = valid_pred
            models.append(model)
    
        feat_imp_df["sum"] = feat_imp_df.iloc[:,1:].sum(axis=1)
        val_score = self.config["metric_function"](self.config["target"], valid_output)
    
        return models, valid_output, feat_imp_df
        
        
    def lgb_objective(self, trial):
        # https://lightgbm.readthedocs.io/en/latest/Parameters.html
        check_params = {
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 10),
            'lambda_l2': trial.suggest_float('lambda_l2', 0, 0.1),
            'lambda_l1': trial.suggest_float('lambda_l1', 0, 0.1),
        }
    
        params = self.config["lgb"]["params"].copy()
        params.update(check_params)
        _, val_output, _ = self.cv_execute("lgb")
        score = self.config["metric_function"](self.config["target"], val_output)
        return score

    
    def xgb_objective(self, trial):
        # https://xgboost.readthedocs.io/en/stable/parameter.html
        check_params = {
            'gamma': trial.suggest_float('gamma', 0, 1),
            'max_depth': trial.suggest_int('max_depth', 1, 10),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
            'subsample': trial.suggest_float('subsample', 0, 1),
            'lambda': trial.suggest_float('lambda', 0, 1),
            'alpha': trial.suggest_float('alpha', 0, 1),
        }
    
        params = self.config["xgb"]["params"].copy()
        params.update(check_params)
        _, val_output, _ = self.cv_execute("xgb")
        score = self.config["metric_function"](self.config["target"], val_output)
        return score

    
    def cb_objective(self, trial):
        # https://catboost.ai/en/docs/references/training-parameters/
        check_params = {
            'max_depth': trial.suggest_int('max_depth', 1, 10),
            #'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
            'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.5),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 10),
            'colsample_bylevel': trial.suggest_float('reg_lambda', 0, 1.0),
        }
    
        params = self.config["cb"]["params"].copy()
        params.update(check_params)
        _, val_output, _ = self.cv_execute("cb")
        score = self.config["metric_function"](self.config["target"], val_output) 
        return score

    
    def param_tuning(self, model_name, option = "minimize"):

        if model_name == "lgb":
            objective = self.lgb_objective
        elif model_name == "xgb":
            objective = self.xgb_objective
        elif model_name == "cb":
            objective = self.cb_objective
            
        study = optuna.create_study(direction=option) 
        study.optimize(objective, n_trials=self.config["optuna_trial_num"])
        trial = study.best_trial
        print('Value: ', trial.value)
        return trial.params

# preprocess

In [3]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')


train['EJ'] = train['EJ'].map({'A': 0, 'B': 1})
test['EJ']  = test['EJ'].map({'A': 0, 'B': 1})

# process epsilon
train = pd.merge(train, greeks, on = "Id", how = "inner")
train_stratify = train[["Class", "Beta", "Delta", "Gamma"]] 
train["Epsilon_ordinal"] = train["Epsilon"].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal() if x != "Unknown" else np.nan)

org_features = [n for n in train.columns if n not in ['Class', 'Id', 'Alpha', "Beta", "Gamma", "Delta", "Epsilon"]]
test_times = pd.DataFrame([train.Epsilon_ordinal.max() + 1] * len(test), columns = ["Epsilon_ordinal"])
final_test = pd.concat((test, test_times), axis=1)

# fill missing value
train.fillna(-999, inplace=True)
final_test.fillna(-999, inplace=True)

# add pca columns
pca_feat_num = 15
pca_cols = ["pca"+str(i+1) for i in range(pca_feat_num)]
pca = PCA(n_components=pca_feat_num,random_state=42)
pca_train = pca.fit_transform(train[org_features])
pca_test = pca.transform(final_test[org_features])
pca_train = pd.DataFrame(pca_train, columns=pca_cols)
pca_test = pd.DataFrame(pca_test, columns=pca_cols)
train = pd.concat([train, pca_train],axis=1)
final_test = pd.concat([final_test, pca_test],axis=1)

print(train.shape, final_test.shape)

(617, 79) (5, 73)


# config

In [4]:
from sklearn.metrics import log_loss
def balanced_log_loss(y_true, y_pred):
    nc = np.bincount(y_true)
    return log_loss(y_true, y_pred, sample_weight = 1/nc[y_true], eps=1e-15)

config = {
    "xgb": {
        "feats": org_features + pca_cols,
        "num_round" : 1000,
        "es_round" : 50,
        "verbose_eval": 100,
        "params" : {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'verbosity': 0,
            'seed': 42
        },   
    },
    
    "lgb": {
        "feats": org_features + pca_cols,
        "num_round" : 1000,
        "es_round" : 20,
        "verbose_eval": 100,
        "params" : {
            'objective': 'binary', 
            'metric': 'binary_logloss', 
            'boosting': 'goss',
            'verbose': -1,
            'seed': 42,
            'is_unbalance': True
        },   
    },

    "cb": {
        "feats": org_features + pca_cols,
        "es_round" : 40,
        "verbose_eval": 100,
        "params" : {
            'loss_function': 'Logloss',
            'iterations': 1000,  
            'random_seed': 42,
            'use_best_model' : True
        },
    },
    "task_type": "classification",
    "train_data": train,
    "test_data": final_test,
    "target": train.Class,
    "metric_function" : balanced_log_loss,
    "optuna_trial_num": 5,
}

# tree model

In [5]:
treemodel = TreeModelling(config)

In [6]:
lgb_models, lgb_val_output, lgb_imp_df = treemodel.cv_execute("lgb")
lgb_test_output = treemodel.lgb_test(lgb_models, "lgb", config["test_data"])
    
lgb_best_params = treemodel.param_tuning("lgb")
lgb_best_params

Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds


[I 2023-10-28 08:04:54,115] A new study created in memory with name: no-name-3073b11f-975e-46e1-b1fa-7205cedecf56


Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds


[I 2023-10-28 08:04:55,654] Trial 0 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 14, 'learning_rate': 0.11175250199807718, 'feature_fraction': 0.829604969318658, 'bagging_fraction': 0.633717918209397, 'min_child_samples': 1, 'lambda_l2': 0.017037482232823697, 'lambda_l1': 0.026677941724291367}. Best is trial 0 with value: 0.22179991403414676.


Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds


...

[I 2023-10-28 08:05:01,160] Trial 3 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 100, 'learning_rate': 0.24624755801345755, 'feature_fraction': 0.7140248377279224, 'bagging_fraction': 0.6131965353947058, 'min_child_samples': 8, 'lambda_l2': 0.06134001363684554, 'lambda_l1': 0.052598097307121595}. Best is trial 0 with value: 0.22179991403414676.


Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds


[I 2023-10-28 08:05:02,741] Trial 4 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 31, 'learning_rate': 0.2899404189870847, 'feature_fraction': 0.9435988965486981, 'bagging_fraction': 0.6476353009408539, 'min_child_samples': 9, 'lambda_l2': 0.0809544699976179, 'lambda_l1': 0.014637676243739973}. Best is trial 0 with value: 0.22179991403414676.


Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839
Value:  0.22179991403414676


{'num_leaves': 14,
 'learning_rate': 0.11175250199807718,
 'feature_fraction': 0.829604969318658,
 'bagging_fraction': 0.633717918209397,
 'min_child_samples': 1,
 'lambda_l2': 0.017037482232823697,
 'lambda_l1': 0.026677941724291367}

In [7]:
xgb_models, xgb_val_output, xgb_imp_df = treemodel.cv_execute("xgb")
xgb_test_output = treemodel.xgb_test(xgb_models, "xgb", config["test_data"])
    
xgb_best_params = treemodel.param_tuning("xgb")
xgb_best_params

[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[294]	eval-logloss:0.07731
[0]	eval-logloss:0.52655
[79]	eval-logloss:0.15150
[0]	eval-logloss:0.50637
[73]	eval-logloss:0.16337
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[145]	eval-logloss:0.15881
[0]	eval-logloss:0.52212
[72]	eval-logloss:0.20594


[I 2023-10-28 08:05:05,091] A new study created in memory with name: no-name-50823bd7-9ae5-4013-8cc0-9d92f4cfccdc


[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[293]	eval-logloss:0.07732
[0]	eval-logloss:0.52655
[79]	eval-logloss:0.15150
[0]	eval-logloss:0.50637
[73]	eval-logloss:0.16337
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[145]	eval-logloss:0.15881
[0]	eval-logloss:0.52212
[72]	eval-logloss:0.20594


[I 2023-10-28 08:05:07,373] Trial 0 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.8422293930161248, 'max_depth': 2, 'num_leaves': 33, 'learning_rate': 0.019507374315956342, 'min_child_weight': 3, 'max_delta_step': 10, 'subsample': 0.6722088515650636, 'lambda': 0.1031093524702934, 'alpha': 0.12577907243745678}. Best is trial 0 with value: 0.31216516835643554.


[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[294]	eval-logloss:0.07731
[0]	eval-logloss:0.52655
[79]	eval-logloss:0.15150
[0]	eval-logloss:0.50637
[73]	eval-logloss:0.16337
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[145]	eval-logloss:0.15881
[0]	eval-logloss:0.52212
[72]	eval-logloss:0.20594


...

[I 2023-10-28 08:05:14,242] Trial 3 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.9861283858592306, 'max_depth': 7, 'num_leaves': 87, 'learning_rate': 0.2391529930985339, 'min_child_weight': 6, 'max_delta_step': 9, 'subsample': 0.5312022404352947, 'lambda': 0.15412928089571154, 'alpha': 0.8257653045449431}. Best is trial 0 with value: 0.31216516835643554.


[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[294]	eval-logloss:0.07731
[0]	eval-logloss:0.52655
[78]	eval-logloss:0.15152
[0]	eval-logloss:0.50637
[74]	eval-logloss:0.16255
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[144]	eval-logloss:0.15872
[0]	eval-logloss:0.52212
[73]	eval-logloss:0.20678


[I 2023-10-28 08:05:16,507] Trial 4 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.29354268805479, 'max_depth': 8, 'num_leaves': 88, 'learning_rate': 0.33863422699768286, 'min_child_weight': 8, 'max_delta_step': 7, 'subsample': 0.17816145616391255, 'lambda': 0.7821532832458215, 'alpha': 0.623965524662699}. Best is trial 0 with value: 0.31216516835643554.


Value:  0.31216516835643554


{'gamma': 0.8422293930161248,
 'max_depth': 2,
 'num_leaves': 33,
 'learning_rate': 0.019507374315956342,
 'min_child_weight': 3,
 'max_delta_step': 10,
 'subsample': 0.6722088515650636,
 'lambda': 0.1031093524702934,
 'alpha': 0.12577907243745678}

In [8]:
cb_models, cb_val_output, cb_imp_df = treemodel.cv_execute("cb")
cb_test_output = treemodel.cb_test(cb_models, "cb", config["test_data"])
   
cb_best_params = treemodel.param_tuning("cb")
cb_best_params

Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 69.3ms	remaining: 1m 9s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 853ms	remaining: 7.59s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.64s	remaining: 6.51s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.41s	remaining: 5.59s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.18s	remaining: 4.75s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 3.97s	remaining: 3.95s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.76s	remaining: 3.16s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.61s	remaining: 2.39s
800:	learn: 0.0042617	test: 0.0891180	best: 0.0891180 (800)	total: 6.4s	remaining: 1.59s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.08903141659
bestIteration = 803

Shrink model to first 804 iterations.
Learning rate set to 0.02

[I 2023-10-28 08:05:37,510] A new study created in memory with name: no-name-f09fd61c-6e32-41ab-a1c2-463f9ae6934d


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.42s	remaining: 5.61s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 10.1ms	remaining: 10.1s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 820ms	remaining: 7.3s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.62s	remaining: 6.43s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.42s	remaining: 5.63s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.21s	remaining: 4.8s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 4.02s	remaining: 4s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.88s	remaining: 3.24s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.68s	remaining: 2.42s
800:	learn: 0.0042617	test: 0.

[I 2023-10-28 08:05:58,213] Trial 0 finished with value: 0.3645940857907282 and parameters: {'max_depth': 6, 'reg_lambda': 0.5633452570338364, 'learning_rate': 0.47579621420884, 'min_child_samples': 3}. Best is trial 0 with value: 0.3645940857907282.


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.44s	remaining: 5.66s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 8.45ms	remaining: 8.44s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 816ms	remaining: 7.26s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.62s	remaining: 6.44s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.42s	remaining: 5.63s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.31s	remaining: 4.95s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 4.48s	remaining: 4.46s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 5.35s	remaining: 3.55s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 6.15s	remaining: 2.62s
800:	learn: 0.0042617	tes

...

[I 2023-10-28 08:07:01,348] Trial 3 finished with value: 0.3645940857907282 and parameters: {'max_depth': 5, 'reg_lambda': 0.4749091678430386, 'learning_rate': 0.47447050607351293, 'min_child_samples': 8}. Best is trial 0 with value: 0.3645940857907282.


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.44s	remaining: 5.67s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 8.25ms	remaining: 8.24s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 884ms	remaining: 7.86s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.72s	remaining: 6.82s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.52s	remaining: 5.86s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.33s	remaining: 4.97s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 4.21s	remaining: 4.19s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 5.31s	remaining: 3.52s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 6.18s	remaining: 2.63s
800:	learn: 0.0042617	tes

[I 2023-10-28 08:07:22,615] Trial 4 finished with value: 0.3645940857907282 and parameters: {'max_depth': 8, 'reg_lambda': 0.7758913929011702, 'learning_rate': 0.07670346153225255, 'min_child_samples': 5}. Best is trial 0 with value: 0.3645940857907282.


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.5s	remaining: 5.82s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
Value:  0.3645940857907282


{'max_depth': 6,
 'reg_lambda': 0.5633452570338364,
 'learning_rate': 0.47579621420884,
 'min_child_samples': 3}